<a href="https://colab.research.google.com/github/MarkusStefan/Data_Analytics/blob/main/AssociateRuleLearning_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [29]:
retail = pd.read_csv("https://raw.githubusercontent.com/MarkusStefan/Data_Analytics/main/Exercise2/retail2.csv", sep=';')
retail.describe()
retail[retail[['Bread', 'Yogurt']]==1]==True

retail[["Bread", "Yogurt"]==1]
#retail.query("(Bread ")

KeyError: ignored

## Support
It indicates how frequently an item set appears in the data set.

Let $frq(A)$ be the number of occurrences (frequency) from the total number of transactions $frq(T) = T$:

$$
supp(A) = \frac{frq(A)}{T}
$$

In [41]:
def support(df, cols):
  if type(cols) != list:
    cols = [cols]
  bool_mask = df[cols].isin([1]).any(axis=1)
  frqA = len(df[bool_mask])
  T = len(df[cols])
  return frqA/T

support(retail, ["Bread", "Yogurt"])
support(retail,'Bread')
  

0.67

## Confidence
It says how likely item set B is purchased when item set A is purchased

$$
conf(A \to B) = \frac{supp(A,B)}{supp(A)}
$$

with $supp(A,B) = \frac{frq(A,B)}{frq(T)}$ and $supp(A) = \frac{frq(A)}{frq(T)}$ it holds that:

$$
conf(A \to B) = \frac{supp(A,B)}{supp(A)} = 
\frac{\frac{frq(A,B)}{frq(T)}}{\frac{frq(A)}{frq(T)}} = 
\frac{frq(A,B)}{frq(A)}
$$

In [20]:
def support1(df, col):
    bool_mask = df[col].isin([1]).any(axis=1)
    frqA = len(df[bool_mask])
    T = len(df[col])
    return frqA/T

support1(retail, ["Bread", 'Yogurt'])

1.0

## Lift
Lift: It says how likely item set B is purchased when item set A is purchased while controlling for how popular item set B is.

Lift is the ratio of the observed support to that expected if A and B were independent or equivalently the ratio of the confidence of the rule to the expected confidence of the RHS item set by independence.

$$
lift(A \to B) = \frac{conf(A,B)}{supp(B)} = \frac{supp(A,B)}{supp(A) \times supp(B)}
$$

Note:

$$
lift(A \to B) == lift(B \to A)
$$